# Merging tables

## Packages

In [1]:
import pandas as pd
import numpy as np

## Import data

In [6]:
contractPath = 'all_data_cleaned.csv'
contract = pd.read_csv(contractPath)

C:\Users\franc\AppData\Local\Temp\ipykernel_18152\2752683583.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  contract = pd.read_csv(contractPath)


In [8]:
damagePath = '../data/damage_clean.csv'
damage = pd.read_csv(damagePath)

C:\Users\franc\AppData\Local\Temp\ipykernel_18152\3551238721.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  damage = pd.read_csv(damagePath)


## Compute pipe premium amount

In [9]:
# Calculate NET_PREMIUM_AMOUNT as 60% of PREMIUM_AMOUNT
contract['NET_PREMIUM_AMOUNT'] = contract['PREMIUM_AMOUNT'] * 0.6

In [10]:
# Calculate PIPE_PREMIUM_AMOUNT based on CORPORATE_DEVISION
# Map devision percentages
devisionPercentages = {
    'VGV': 0.43,
    'VHV': 0.11,
    'H&H': 0.07,
    'W&W': 0.32
}

# Calculate PIPE_PREMIUM_AMOUNT
contract['PIPE_PREMIUM_AMOUNT'] = contract['CORPORATE_DEVISION'].map(devisionPercentages) * contract['NET_PREMIUM_AMOUNT']

In [11]:
# Drop columns we don't need
contract = contract.drop(columns=['NET_PREMIUM_AMOUNT', 'PREMIUM_AMOUNT'])

## Create damage variable

In [13]:
# Create varaible in the damage table, always equal to 1
damage['DAMAGE'] = 1

In [20]:
# Convert dates to date format
damage['DAT_EREIGNIS'] = pd.to_datetime(damage['DAT_EREIGNIS'], errors='coerce', infer_datetime_format=True)
contract['contract_year'] = pd.to_datetime(contract['contract_year'], errors='coerce', infer_datetime_format=True)

# Extract year from damage and contract data
damage['YEAR'] = damage['DAT_EREIGNIS'].dt.year
contract['YEAR'] = contract['contract_year'].dt.year

C:\Users\franc\AppData\Local\Temp\ipykernel_18152\2302168266.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  damage['DAT_EREIGNIS'] = pd.to_datetime(damage['DAT_EREIGNIS'], errors='coerce', infer_datetime_format=True)
C:\Users\franc\AppData\Local\Temp\ipykernel_18152\2302168266.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  contract['contract_year'] = pd.to_datetime(contract['contract_year'], errors='coerce', infer_datetime_format=True)


In [21]:
# Add damage column to contract data
contract = pd.merge(contract, damage[['DAMAGE', 'ANO_SID', 'YEAR']], left_on=['ANO_SID', 'YEAR'], right_on=['ANO_SID', 'YEAR'], how='left')

# Replace NaNs in the 'damage' column with 0
contract['DAMAGE'] = contract['DAMAGE'].fillna(0)

# Convert the 'damage' column to int16
contract['DAMAGE'] = contract['DAMAGE'].astype('int16')

In [ ]:
# Save dataset to csv
contract.to_csv('contract_classification.csv', index=False)

In [ ]:
contr